In [1]:
import wandb
from tsai.all import *

In [27]:
BS = 64

In [28]:
api = wandb.Api()
artifact = api.artifact('manan-goel/PhysioNet_Challenge/preprocessed_dataset:v0')
artifact.download()

wandb: Downloading large artifact preprocessed_dataset:v0, 3148.19MB. 2 files... Done. 0:0:0


'./artifacts/preprocessed_dataset:v0'

In [29]:
dataset_path = Path('./artifacts/preprocessed_dataset:v0')

In [30]:
data = np.load(dataset_path/"downsampled_signals.npy")

In [31]:
data.shape

(6877, 12, 5000)

In [32]:
df = pd.read_csv(dataset_path/"labels.csv", index_col=0)

In [33]:
X, y = data, df.values

In [34]:
tfms  = [None, TSClassification()] # TSClassification == Categorize
batch_tfms = TSStandardize()

In [35]:
assert len(X) == len(y), f"Should {len(X)=} == {len(y)=}"

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
# def split(X, pct):
N = len(X)
idxs = range_of(N)

In [38]:
split = train_test_split(idxs, test_size=1000)

In [39]:
dls = get_ts_dls(X, 
                 y, 
                 splits=split,
                 tfms=tfms, 
                 batch_tfms=batch_tfms, 
                 bs=BS)

In [40]:
x,y = dls.one_batch()

In [41]:
x.shape, y.shape

(torch.Size([64, 12, 5000]), torch.Size([64, 9]))

In [42]:
model = InceptionTime(12, 9).cuda()

In [43]:
out = model(x.cuda())

In [44]:
def my_accuracy_multi(inp, targ, thresh=0.5, sigmoid=True, by_sample=False):
    "Computes accuracy when `inp` and `targ` are the same size."
    if sigmoid: inp = inp.sigmoid()
    correct = (inp>thresh)==targ.bool()
    if by_sample:
        return (correct.float().mean(-1) == 1).float().mean()
    else:
        inp,targ = flatten_check(inp,targ)
        return correct.float().mean()

In [45]:
my_accuracy_multi(out, y)

TensorCategory(0.5538, device='cuda:0')

In [46]:
from fastai.callback.fp16 import MixedPrecision
from fastai.callback.wandb import WandbCallback

In [47]:
wandb.init(entity="timeseriesbois", project="tsai")

wandb: wandb version 0.12.13 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [48]:
wandb_cb = WandbCallback(log_preds=False, log_model=False)

In [49]:
learn = Learner(dls, 
                model, 
                loss_func=BCEWithLogitsLossFlat(), 
                metrics=[my_accuracy_multi],
                cbs=[wandb_cb, MixedPrecision()])

In [ ]:
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,my_accuracy_multi,time
0,0.491945,0.363853,0.901889,00:08
1,0.299240,0.255680,0.909889,00:08
2,0.238441,0.214254,0.923667,00:08
3,0.209513,0.234816,0.915111,00:08
4,0.193131,0.203904,0.924778,00:08
5,0.177594,0.176413,0.937667,00:08
6,0.165057,0.169570,0.940000,00:08
7,0.154018,0.160415,0.944111,00:08


In [ ]:
wandb.finish()

## Predictions

In [ ]:
preds = learn.get_preds( )